In [115]:
import deepchem as dc
import pandas as pd
import pickle
import numpy as np
import warnings
import tensorflow as tf
import math
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from utils import *
warnings.filterwarnings('ignore')
file = open('tests/null/model_10/results.pickle','rb')
results = pickle.load(file)
stats = results['stats'][0]
stats

{'phy_rmsd': {'train': 5.4019556726215345, 'test': 5.44957314842181},
 'ml_rmsd': {'train': 1.2810525074071437, 'test': 1.814279391253112},
 'phy_md': {'train': -3.7889501779359427, 'test': -3.90175},
 'ml_md': {'train': -0.8047862237447946, 'test': -0.9768791412860154},
 'phy_out_rmsd': {'train': 14.264721252579493, 'test': 14.343293729126515},
 'ml_out_rmsd': {'train': 3.4633501477309445, 'test': 5.848452018417741},
 'params': {'epochs': 500,
  'dropout': 0.4,
  'batch_normalize': False,
  'batch_size': 100,
  'feat': 'null',
  'kfold': 20,
  'dense_layer_size': 27,
  'graph_conv_layers': [32, 32],
  'model_dir': 'model_10'}}

In [94]:
df=pd.read_csv('dicts/full_dataset.csv')
df_test=pd.read_csv('dicts/test_dataset.csv')
df_train=pd.read_csv('dicts/train_dataset.csv')
df_test
# rmsd(df_test['expt'],df_test['gbnsr6'])

,id,smiles,iupac name,expt,tip3p,cha,gbnsr6,asc,igb5,null
0,mobley_8525830,CCCCCI,1-iodopentane,-0.14,-0.111,-0.599923,0.953224,-0.433,-1.550,0
1,mobley_172879,c1ccc2c(c1)Oc3ccccc3O2,dibenzo-p-dioxin,-3.15,-4.900,-3.978600,-6.237540,-4.664,-6.369,0
2,mobley_1708457,CS(=O)(=O)C,methylsulfonylmethane,-10.08,-10.559,-11.459300,-12.683500,-14.392,-15.336,0
3,mobley_2481002,C([N+](=O)[O-])(Cl)(Cl)Cl,trichloro(nitro)methane,-1.45,-0.379,-0.516043,-0.198253,1.789,-0.164,0
4,mobley_2844990,c1cc2c(cc1Cl)Oc3cc(c(c(c3O2)Cl)Cl)Cl,"1,2,3,7-tetrachlorodibenzo-p-dioxin",-3.84,-2.660,-2.403790,-3.618940,-0.367,-2.501,0
...,...,...,...,...,...,...,...,...,...,...
75,mobley_1017962,CCCCCC(=O)OC,methyl hexanoate,-2.49,-3.299,-3.726220,-3.004340,-3.886,-5.059,0
76,mobley_1857976,C(CCl)Cl,"1,2-dichloroethane",-1.79,-0.363,-2.108960,-2.243750,-2.122,-2.904,0
77,mobley_5600967,CN(C)C(=O)c1ccc(cc1)OC,"4-methoxy-N,N-dimethyl-benzamide",-11.01,-9.625,-9.868140,-9.223820,-11.153,-11.794,0
78,mobley_6248915,CCCCCCCCN,octan-1-amine,-3.65,-2.589,-1.732710,-0.007307,0.783,-2.282,0


In [95]:
params = {'epochs' : 500, 'dropout' : 0.4, 'batch_normalize' : False, 'batch_size' : 100, 
           'dense_layer_size' : 27, 'graph_conv_layers' : [32, 32]}

In [110]:
# import tensorflow as tfd
# tf.get_logger().setLevel('ERROR')
def test_reload_model(feat,i,stats):
    reloaded_model = dc.models.GraphConvModel(n_tasks=1, graph_conv_layers=params['graph_conv_layers'],
                                     mode='regression', dropout=params['dropout'], 
                                     batch_normalize=params['batch_normalize'], 
                                     batch_size=params['batch_size'], 
                                     dense_layer_size=params['dense_layer_size'],
                                     model_dir = 'tests/'+feat+'/model_'+str(i)
                                )
    reloaded_model.restore()#.expect_partial()
    featurizer = dc.feat.ConvMolFeaturizer(per_atom_fragmentation=False)
    train_X = featurizer.featurize(df_train['smiles'])
    test_X = featurizer.featurize(df_test['smiles'])
    train_y_hat = np.array(reloaded_model.predict_on_batch(train_X)).flatten()
    test_y_hat = np.array(reloaded_model.predict_on_batch(test_X)).flatten()
    p_rmsd_train = rmsd(df_train['expt'],df_train[feat].to_numpy())
    p_rmsd_test=rmsd(df_test['expt'],df_test[feat].to_numpy())
    ml_rmsd_train=rmsd(df_train['expt'],df_train[feat].to_numpy()+train_y_hat)
    ml_rmsd_test=rmsd(df_test['expt'],df_test[feat].to_numpy()+test_y_hat)
    if(not math.isclose(stats['phy_rmsd']['test'], p_rmsd_test,rel_tol=tol) or not math.isclose(stats['phy_rmsd']['train'],p_rmsd_train,rel_tol=tol)
          or not math.isclose(stats['ml_rmsd']['test'], ml_rmsd_test,rel_tol=tol) or not math.isclose(stats['ml_rmsd']['train'], ml_rmsd_train,rel_tol=tol)):
            print('error:',stats['phy_rmsd']['test'], rphys_test, stats['phy_rmsd']['train'], rphys_train
          ,stats['ml_rmsd']['test'], rml_test ,stats['ml_rmsd']['train'], rml_train)
    # return p_rmsd_train,p_rmsd_test,ml_rmsd_train,ml_rmsd_test

In [144]:
final_results = pd.DataFrame(columns=['physics_model', 'phy_rmsd_X', 'phy_rmsd_y',
                                        'phy_md_X','phy_md_y',
                                        'phy_ormsd_X','phy_ormsd_y',
                                        'ml_rmsd_X','ml_rmsd_X_std', 'ml_rmsd_y','ml_rmsd_y_std',
                                        'ml_md_X','ml_md_X_std','ml_md_y','ml_md_y_std',
                                        'ml_ormsd_X','ml_ormsd_X_std','ml_ormsd_y','ml_ormsd_y_std',
                                        'final_model_index','final_ml_rmsd_X','final_ml_rmsd_y',
                                        'final_ml_md_X','final_ml_md_y',
                                        'final_ml_ormsd_X','final_ml_ormsd_y'
                                     ])
train_cols=['physics_model', 'phy_rmsd_X','phy_md_X',
            'phy_ormsd_X','ml_rmsd_X','ml_rmsd_X_std',
            'ml_md_X','ml_md_X_std','ml_ormsd_X','ml_ormsd_X_std']
test_cols=['physics_model', 'phy_rmsd_y','phy_md_y',
            'phy_ormsd_y','ml_rmsd_y','ml_rmsd_y_std',
            'ml_md_y','ml_md_y_std','ml_ormsd_y','ml_ormsd_y_std']
final_cols=['final_ml_rmsd_X','final_ml_rmsd_y',
            'final_ml_md_X','final_ml_md_y',
            'final_ml_ormsd_X','final_ml_ormsd_y']
final_results[train_cols]

,physics_model,phy_rmsd_X,phy_md_X,phy_ormsd_X,ml_rmsd_X,ml_rmsd_X_std,ml_md_X,ml_md_X_std,ml_ormsd_X,ml_ormsd_X_std


In [145]:
# import tensorflow as tf
# tf.get_logger().setLevel('ERROR')
error = 1
r = 4
tol = 1e-08

warnings.filterwarnings('ignore')
feats = ['tip3p','cha','gbnsr6','asc','igb5','null']
for feat in feats:
    print(feat)
    phys_rmsd_train,phys_rmsd_test,ml_rmsd_train,ml_rmsd_test = [],[],[],[]
    phys_md_test,phys_md_train,ml_md_test,ml_md_train = [],[],[],[]
    phys_ormsd_test, phys_ormsd_train, ml_ormsd_test,ml_ormsd_train= [],[],[],[]
    for i in range(100):
        file = open('tests/'+feat+'/model_'+str(i)+'/results.pickle','rb')
        # print(rphys_train,rphys_test,rml_train,rml_test)
        results = pickle.load(file)
        stats = results['stats'][0]
        
        phys_rmsd_test.append(stats['phy_rmsd']['test'])
        phys_rmsd_train.append(stats['phy_rmsd']['train'])
        ml_rmsd_test.append(stats['ml_rmsd']['test'])
        ml_rmsd_train.append(stats['ml_rmsd']['train'])
        phys_md_test.append(stats['phy_md']['test'])
        phys_md_train.append(stats['phy_md']['train'])
        ml_md_test.append(stats['ml_md']['test'])
        ml_md_train.append(stats['ml_md']['train'])
        phys_ormsd_test.append(stats['phy_out_rmsd']['test'])
        phys_ormsd_train.append(stats['phy_out_rmsd']['train'])
        ml_ormsd_test.append(stats['ml_out_rmsd']['test'])
        ml_ormsd_train.append(stats['ml_out_rmsd']['train'])
        # test_reload_model(feat,i,stats)
    best = -1
    dif = 1
    avg_train_rmsd = np.mean(ml_rmsd_train)
    for i in range(100):
        file = open('tests/'+feat+'/model_'+str(i)+'/results.pickle','rb')
        results = pickle.load(file)
        stats = results['stats'][0]
        if(abs(stats['ml_rmsd']['train']-avg_train_rmsd) < dif):
            best = i
            dif = abs(stats['ml_rmsd']['train']-avg_train_rmsd)
    print(best)
    file = open('tests/'+feat+'/model_'+str(best)+'/results.pickle','rb')
    results = pickle.load(file)
    stats = results['stats'][0]
    print('physics model: test',stats['phy_rmsd']['test'],'train',stats['phy_rmsd']['train'])
    print('physics + ml: test',stats['ml_rmsd']['test'],'train',stats['ml_rmsd']['train'])
    
    final_results.loc[len(final_results.index)] = [feat,np.mean(phys_rmsd_train),np.mean(phys_rmsd_test),
                  np.mean(phys_md_train),np.mean(phys_md_test),np.mean(phys_ormsd_train),np.mean(phys_ormsd_test),
                  np.mean(ml_rmsd_train),np.std(ml_rmsd_train),np.mean(ml_rmsd_test),np.std(ml_rmsd_test),
                  np.mean(ml_md_train),np.std(ml_md_train),np.mean(ml_md_test),np.std(ml_md_test),
                  np.mean(ml_ormsd_train),np.std(ml_ormsd_train),np.mean(ml_ormsd_test),np.std(ml_ormsd_test),
                  best,stats['ml_rmsd']['train'],stats['ml_rmsd']['test'],stats['ml_md']['train'],
                  stats['ml_md']['test'],stats['ml_out_rmsd']['train'],stats['ml_out_rmsd']['test']
                 ]
    print('physics model: test',np.round(np.mean(phys_rmsd_test),r),'±',np.round(error*np.std(phys_rmsd_test),r),
              'train',np.round(np.mean(phys_rmsd_train),r),'±',np.round(error*np.std(phys_rmsd_train),r))
    print('physics + ml: test',np.round(np.mean(ml_rmsd_test),r),'±',np.round(error*np.std(ml_rmsd_test),r),
              'train',np.round(np.mean(ml_rmsd_train),r),'±',np.round(error*np.std(ml_rmsd_train),r))
final_results[final_cols]

tip3p
79
physics model: test 1.4352081556345755 train 1.5560596697201357
physics + ml: test 1.0005259458989328 train 0.7070355815417102
physics model: test 1.4352 ± 0.0 train 1.5561 ± 0.0
physics + ml: test 1.0381 ± 0.0711 train 0.7065 ± 0.0477
cha
43
physics model: test 1.3152918288589681 train 1.7703504244090553
physics + ml: test 0.979833716526537 train 0.774470910833617
physics model: test 1.3153 ± 0.0 train 1.7704 ± 0.0
physics + ml: test 0.9914 ± 0.058 train 0.7749 ± 0.0396
gbnsr6
26
physics model: test 1.8545443800079835 train 1.6401879965014992
physics + ml: test 1.5061749198632763 train 0.7563507809473902
physics model: test 1.8545 ± 0.0 train 1.6402 ± 0.0
physics + ml: test 1.4592 ± 0.0718 train 0.756 ± 0.0459
asc
83
physics model: test 2.6515044880218475 train 2.4890836992599725
physics + ml: test 1.5951079405883095 train 1.0411860286306023
physics model: test 2.6515 ± 0.0 train 2.4891 ± 0.0
physics + ml: test 1.5728 ± 0.0957 train 1.0382 ± 0.0614
igb5
1
physics model: test 

,final_ml_rmsd_X,final_ml_rmsd_y,final_ml_md_X,final_ml_md_y,final_ml_ormsd_X,final_ml_ormsd_y
0,0.707036,1.000526,0.149158,0.297814,2.262706,3.016019
1,0.774471,0.979834,0.076602,0.099154,2.466486,2.943363
2,0.756351,1.506175,0.181459,0.410187,2.376280,5.052698
3,1.041186,1.595108,0.018136,0.171691,3.453873,4.245788
4,1.039211,1.716728,0.155193,0.372257,3.383347,5.517370
5,1.122022,1.601314,-0.608265,-0.701710,3.195818,4.990649


In [132]:
final_results.to_csv('dicts/final_results.csv',index=False)